In [30]:
import operator

In [93]:
!jupyter nbextension enable --py --sys-prefix widgetsnbextension
def log_progress(sequence, title='', every=1, size=None, end_msg='', removeAtTheEnd=False):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{index} / ?'.format(index=index)
                else:
                    progress.value = index
                    if isinstance(sequence, dict):
                        val = ''
                    else:
                        val=sequence[index-1]
                            
                    label.value = u'{title} : {value} ({index} / {size})'.format(
                        title=title,
                        value=val,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = end_msg
        if removeAtTheEnd:
            box.close()
    

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [2]:
import os
import torch
import numpy as np
from datetime import datetime

from faster_rcnn import network
from faster_rcnn.faster_rcnn import FasterRCNN, RPN
from faster_rcnn.utils.timer import Timer

import faster_rcnn.roi_data_layer.roidb as rdl_roidb
from faster_rcnn.roi_data_layer.layer import RoIDataLayer
from faster_rcnn.datasets.factory import get_imdb
from faster_rcnn.fast_rcnn.config import cfg, cfg_from_file

try:
    from termcolor import cprint
except ImportError:
    cprint = None

try:
    from pycrayon import CrayonClient
except ImportError:
    1
CrayonClient = None


def log_print(text, color=None, on_color=None, attrs=None):
    if cprint is not None:
        cprint(text, color=color, on_color=on_color, attrs=attrs)
    else:
        print(text)



# hyper-parameters
# ------------

pytorchpath = os.environ['PWD']+'/'
# print pytorchpath
# exit()
# pytorchpath = '/data02/jguerry/jg_pyt/'
pytorchpath = '/home/jguerry/workspace/jg_dl/jg_pyt/'


imdb_name = 'sunrgbd_train_rgb_i_100_8bits'
output_dir = pytorchpath+'models/'+imdb_name+'_noChairs/'

cfg_file = pytorchpath+'experiments/cfgs/faster_rcnn_end2end_sunrgbd.yml'
pretrained_model = pytorchpath+'data/pretrained_model/VGG_imagenet.npy'
start_step = 0
end_step = 100000
lr_decay_steps = {60000, 80000}
lr_decay = 1./10

rand_seed = 1024
_DEBUG = True
use_tensorboard = True
remove_all_log = False   # remove all historical experiments in TensorBoard
exp_name = None # the previous experiment name in TensorBoard

# ------------

if rand_seed is not None:
    np.random.seed(rand_seed)

# load config
cfg_from_file(cfg_file)
lr = cfg.TRAIN.LEARNING_RATE
momentum = cfg.TRAIN.MOMENTUM
weight_decay = cfg.TRAIN.WEIGHT_DECAY
disp_interval = cfg.TRAIN.DISPLAY
log_interval = cfg.TRAIN.LOG_IMAGE_ITERS

# load data
imdb = get_imdb(imdb_name)
rdl_roidb.prepare_roidb(imdb)
roidb = imdb.roidb
data_layer = RoIDataLayer(roidb, imdb.num_classes)

# load net
net = FasterRCNN(classes=imdb.classes, debug=_DEBUG)
network.weights_normal_init(net, dev=0.01)
network.load_pretrained_npy(net, pretrained_model)
# model_file = '/media/longc/Data/models/VGGnet_fast_rcnn_iter_70000.h5'
# model_file = 'models/saved_model3/faster_rcnn_60000.h5'
# network.load_net(model_file, net)
# exp_name = 'vgg16_02-19_13-24'
# start_step = 60001
# lr /= 10.
# network.weights_normal_init([net.bbox_fc, net.score_fc, net.fc6, net.fc7], dev=0.01)

net.cuda()
net.train()

params = list(net.parameters())
# optimizer = torch.optim.Adam(params[-8:], lr=lr)
optimizer = torch.optim.SGD(params[8:], lr=lr, momentum=momentum, weight_decay=weight_decay)

if not os.path.exists(output_dir):
    os.mkdir(output_dir)

# tensorboad
use_tensorboard = use_tensorboard and CrayonClient is not None
if use_tensorboard:
    cc = CrayonClient(hostname='127.0.0.1')
    if remove_all_log:
        cc.remove_all_experiments()
    if exp_name is None:
        exp_name = datetime.now().strftime('vgg16_%m-%d_%H-%M')
        exp = cc.create_experiment(exp_name)
    else:
        exp = cc.open_experiment(exp_name)

# training
train_loss = 0
tp, tf, fg, bg = 0., 0., 0, 0
step_cnt = 0
re_cnt = False
t = Timer()
t.tic()

Remove empty annotations:  Done. [ 0 removed]
sunrgbd_train_rgb_i_100_8bits gt roidb reloaded from /c16/THESE.JORIS/datasets/SUNRGBD_pv/faster_rcnn/cache/sunrgbd_train_rgb_i_100_8bits_gt_roidb.pkl


In [14]:
# get one batch
blobs = data_layer.forward()
im_data = blobs['data']
im_info = blobs['im_info']
gt_boxes = blobs['gt_boxes']
gt_ishard = blobs['gt_ishard']
dontcare_areas = blobs['dontcare_areas']
dontcare_areas = None

In [38]:
array = np.array([4,2,7,1])
temp = array.argsort()
ranks = np.empty(len(array), int)
ranks[temp] = np.arange(len(array))

In [39]:
ranks

array([2, 1, 3, 0])

In [43]:
len(sorted_x)

30

In [95]:
t = Timer()
t.tic()

myClassesDict = {}
for i in log_progress(range(1000),title='INDICES', end_msg='Finished !'): 
    # get one batch
    blobs = data_layer.forward()
    im_data = blobs['data']
    im_info = blobs['im_info']
    gt_boxes = blobs['gt_boxes']
    gt_ishard = blobs['gt_ishard']
    dontcare_areas = blobs['dontcare_areas']
    dontcare_areas = None
    
    indices = []

    
    for id_gt,gt in enumerate(gt_boxes):
        ranks = {key: rank for rank, key in enumerate(sorted(myClassesDict, key=myClassesDict.get, reverse=True), 1)}    
        cls=int(gt[4])
        if imdb._classes[cls] not in myClassesDict.keys():
            indices.append(id_gt)
#             myClassesDict[imdb._classes[cls]] = myClassesDict.get(imdb._classes[cls], 0) + 1
        elif ranks[imdb._classes[cls]]>0.6*len(myClassesDict):
            indices.append(id_gt)
#             myClassesDict[imdb._classes[cls]] = myClassesDict.get(imdb._classes[cls], 0) + 1
        else:
            pass
        
    gt_boxes = gt_boxes[indices,:]
    gt_ishard = gt_ishard[indices]
    
    for id_gt,gt in enumerate(gt_boxes):
        cls=int(gt[4])
        myClassesDict[imdb._classes[cls]] = myClassesDict.get(imdb._classes[cls], 0) + 1
       

        
    myClasses_sorted = sorted(myClassesDict.items(), key=operator.itemgetter(1),reverse=True)
            


In [96]:
myClasses_sorted

[('picture', 53),
 ('table', 50),
 ('bookshelf', 50),
 ('books', 50),
 ('pillow', 50),
 ('lamp', 49),
 ('dresser', 49),
 ('sofa', 49),
 ('bed', 49),
 ('paper', 48),
 ('door', 48),
 ('cabinet', 48),
 ('desk', 48),
 ('box', 48),
 ('counter', 48),
 ('whiteboard', 47),
 ('night_stand', 47),
 ('mirror', 47),
 ('person', 47),
 ('tv', 47),
 ('sink', 47),
 ('blinds', 46),
 ('towel', 46),
 ('bag', 45),
 ('curtain', 39),
 ('toilet', 34),
 ('fridge', 33),
 ('bathtub', 24),
 ('window', 7),
 ('shower_curtain', 1)]